In [2]:
import subprocess
import pandas as pd
import os

In [3]:
class color:
    RED = '\033[91m'
    GREEN = '\033[92m'
    END = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

languages = ['C++', 'Java']

In [4]:
def run_exercise(exerciseNo, language, cols, blocksize=0):
    if language == 'C++':
        # check if os is windows
        if os.name == 'nt':
            subprocess.run(["g++", "-O2", "c-implementation/no-papi.cpp", "-o", "matrix"])    
            proc = subprocess.Popen(["./matrix.exe"], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
        
        elif os.name == 'posix':
            subprocess.run(["g++", "-O2", "c-implementation/matrix.cpp", "-o", "matrix", "-lpapi"])
            proc = subprocess.Popen(["./matrix"], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
        
        # Run the program
    
    elif language == 'Java':
        subprocess.run(["javac", "java-implementation/Main.java"])
        proc = subprocess.Popen(["java", "Main"], stdout=subprocess.PIPE, stdin=subprocess.PIPE, cwd="java-implementation")

    # Run the program
    input1 = str(exerciseNo) + "\n"     # Exercise number
    input2 = str(cols) + "\n"           # Size of matrix
    input3 = str(blocksize) + "\n" if exerciseNo == 3 else str(0) + "\n"   # Block size -> is 0 for exercise 1 and 2

    # Feed the input to the program
    proc.stdin.write(input1.encode())
    proc.stdin.flush()

    proc.stdin.write(input2.encode())
    proc.stdin.flush()

    proc.stdin.write(input3.encode())
    proc.stdin.flush()
    
    if exerciseNo == 3:
        # Send the "exit" option
        proc.stdin.write("0\n".encode())
        proc.stdin.flush()

    # wait for the process to finish and get output
    output, err = proc.communicate()

    #  Return the output
    return output.decode()
    

In [5]:
def create_table(blocksize=False):
    table = pd.DataFrame(columns=['Dimensions', 'Language', 'Time (s)']) if not blocksize else pd.DataFrame(columns=['Dimensions', 'Block Size', 'Language', 'Time (s)'])

    # Return the table
    return table

In [ ]:
## EXERCISE 1

# Create the table
table = create_table()
 
results = []

# Run the program for different matrix sizes, 12 times and save the results to array
for cols in range(600, 3001, 400):
    for language in languages:  
        for i in range(12):
            print ("Running iteration no " + str(i+1)  + " in " + color.BOLD + language + color.END + " with dimensions " + str(cols) + "x" + str(cols))

            # Run the exercise
            out = run_exercise(1, language, cols)

            # Get the time
            time = out.lower().split('time')[1].split(' ')[1]

            results.append(time)

        # Remove max and min from the results
        results.remove(max(results))
        results.remove(min(results))

        # Average the results
        avg = sum(map(float, results)) / len(results)

        # Add the results to the table
        table = pd.concat([table, pd.DataFrame([[cols, language, avg]], columns=['Dimensions', 'Language', 'Time (s)'])])

        # Save the results to a csv file, to plot the results later
        results = pd.DataFrame(results, columns=['Time (s)'])
        results.to_csv('runs/ex1/ex1_' + str(cols) + 'x' + str(cols) + '_' + language + '.csv', index=False)

        # Clear the results array
        results = []

# Save the table to a markdown file
table.to_markdown('metrics/ex1/ex1.md', index=False)


In [ ]:
## EXERCISE 2a

# Create the table
table = create_table()

results = []

# Run the program for different matrix sizes
for cols in range(600, 3001, 400):
    for language in languages:  
        for i in range(12):
            print ("Running iteration no " + str(i+1)  + " in " + color.BOLD + language + color.END + " with dimensions " + str(cols) + "x" + str(cols))

            # Run the exercise
            out = run_exercise(2, language, cols)

            # Get the time
            time = out.lower().split('time')[1].split(' ')[1]

            # if java, replace comma with dot
            if language == 'Java':
                time = time.replace(',', '.')

            results.append(time)

        # Remove max and min from the results
        results.remove(max(results))
        results.remove(min(results))

        # Average the results
        avg = sum(map(float, results)) / len(results)
 
        # Add the results to the table
        table = pd.concat([table, pd.DataFrame([[cols, language, avg]], columns=['Dimensions', 'Language', 'Time (s)'])])

        # Save the results to a csv file, to plot the results later
        results = pd.DataFrame(results, columns=['Time (s)'])
        results.to_csv('runs/ex2/ex2_' + str(cols) + 'x' + str(cols) + '_' + language + '.csv', index=False)

        # Clear the results array
        results = []

# Save the table to a markdown file
table.to_markdown('metrics/ex2/ex2-a.md', index=False)




In [ ]:
## EXERCISE 2b

# Run in again for C/C++

## Create the table
table = create_table()

results = []

for cols in range(4096, 10241, 2048):
    for i in range(12):
        print ("Running iteration no " + str(i+1)  + " in " + color.BOLD + "C++" + color.END + " with dimensions " + str(cols) + "x" + str(cols))

        # Run the exercise
        out = run_exercise(2, "C++", cols)

        # Get the time
        time = out.lower().split('time')[1].split(' ')[1]

        results.append(time)

    # Remove max and min from the results
    results.remove(max(results))
    results.remove(min(results))

    # Average the results
    avg = sum(map(float, results)) / len(results)
 
    # Add the results to the table
    table = pd.concat([table, pd.DataFrame([[cols, "C++", avg]], columns=['Dimensions', 'Language', 'Time (s)'])])

    # Save the results to a csv file, to plot the results later
    results = pd.DataFrame(results, columns=['Time (s)'])
    results.to_csv('runs/ex2/ex2_' + str(cols) + 'x' + str(cols) + '_' + "C++" + '.csv', index=False)

    # Clear the results array
    results = []

# Save the table to a markdown file
table.to_markdown('metrics/ex2/ex2-b.md', index=False)

In [6]:
## EXERCISE 3

# Create the table
table = create_table(True)

results = []

# Run the program for different matrix sizes and block sizes
for cols in range(4096, 10241, 2048):
    for blocksize in range(128, 513, 128):
        for i in range(12):
            print ("Running iteration no " + str(i+1) + " in " + color.BOLD + "C++" + color.END + " with dimensions " + str(cols) + "x" + str(cols) + " and block size " + str(blocksize))
            
            # Run the exercise
            out = run_exercise(3, "C++", cols, blocksize)

            # Get the time
            time = out.lower().split('time')[1].split(' ')[1]

            results.append(time)

        # Remove max and min from the results
        results.remove(max(results))
        results.remove(min(results))

        # Average the results
        avg = sum(map(float, results)) / len(results)
    
        # Append a new row to the table
        table = pd.concat([table, pd.DataFrame([[cols, blocksize, "C++", avg]], columns=['Dimensions', 'Block Size', 'Language', 'Time (s)'])])

        # Save the results to a csv file, to plot the results later
        results = pd.DataFrame(results, columns=['Time (s)'])
        results.to_csv('runs/ex3/ex3_' + str(cols) + 'x' + str(cols) + '_' + str(blocksize) + '_' + "C++" + '.csv', index=False)

        # Clear the results array
        results = []
    
    # Save the table to a markdown file
    table.to_markdown('metrics/ex3/ex3-{cols}.md'.format(cols=cols), index=False)

    # Clear the table
    table = create_table(True)

Running iteration no 1 in C++ with dimensions 10240x10240 and block size 512
Running iteration no 2 in C++ with dimensions 10240x10240 and block size 512


In [ ]:
def get_counters(output: str) -> dict:
    """Get the counters from the output of the program"""
    counters = {}

    # Get the counters
    papi_res = output.split("*** PAPI RESULTS ***")[1]

    # Replace tabs with nothing to ease the parsing
    papi_res = papi_res.replace('\t', '')

    # iterate
    for line in papi_res.split('\n'):
        if line == '' or not line.startswith('PAPI'):
            continue

        # Get the counter name and value
        counter_name = line.split(':')[0]
        counter_value = line.split(':')[1].split(' ')[1]

        # Add the counter to the dictionary
        counters[counter_name] = counter_value

    return counters


In [ ]:
def ex1(cols):
    # Get the counters for the first exercise
    output1 = run_exercise(1, "C++", cols)
    counters1 = get_counters(output1) 

    # Create a pd.DataFrame with the counters, whose index is the counter name
    counters1 = pd.DataFrame.from_dict(counters1, orient='index', columns=['Value'])
    counters1.index.name = 'Counter'

    # Save the counters to a csv file
    counters1.to_csv(f'counters/ex1/ex1-{cols}.csv')

In [ ]:
def ex2(cols):
    # Get the counters for the second exercise
    output2 = run_exercise(2, "C++", cols)
    counters2 = get_counters(output2)

    # Create a pd.DataFrame with the counters, whose index is the counter name
    counters2 = pd.DataFrame.from_dict(counters2, orient='index', columns=['Value'])
    counters2.index.name = 'Counter'

    # Save the counters to a csv file
    counters2.to_csv(f'counters/ex2/ex2-{cols}.csv')

In [ ]:
def ex3(cols, blksize):
    
    output3 = run_exercise(3, "C++", cols, blksize)
    counters3 = get_counters(output3)

    # Create a pd.DataFrame with the counters, whose index is the counter name
    counters3 = pd.DataFrame.from_dict(counters3, orient='index', columns=['Value'])

    # Save the counters to a markdown file
    counters3.to_markdown(f'counters/ex3/ex3-{cols}-{blksize}.md')

In [ ]:
for cols in range(600, 3001, 400):
    ex1(cols)
    ex2(cols)
    blksize =  cols // 2
    ex3(cols, blksize)